In [1]:
# Import dependencies for web scraping and data cleaning

import pandas as pd
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://www.newegg.com/d/Best-Sellers/CPUs-Processors/c/ID-34?tid=6676'
browser.visit(url)
#html = browser.html
#soup = BeautifulSoup(html,'html.parser')

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 97.0.4692
[WDM] - Get LATEST driver version for 97.0.4692
[WDM] - Driver [C:\Users\rache\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


In [3]:
# Build a loop to go through each page (splinter) and append cpu names as well as prices into two seperate lists

page_numbers = []
cpu_list = []
price_list = []
x = 0
for x in range (0,9):
    html = browser.html
    soup = BeautifulSoup(html,'html.parser')
    x = x+1
    page_numbers.append(x)
    string_ints = [str(int) for int in page_numbers]
    cpus = soup.find_all("a", class_= 'item-title')
    prices = soup.find_all('li',class_='price-current')
    for cpu in cpus:
        cpu_list.append(cpu.text)
    
        
    for price in prices:
        price_list.append(price.text)
  
    browser.find_by_text(string_ints[x-1]).click()

In [4]:
# Check to see if two lists have the same lenghth

print(cpu_list)
print(price_list)
print(len(cpu_list))
print(len(price_list))

['AMD Ryzen 5 5600X - Ryzen 5 5000 Series Vermeer (Zen 3) 6-Core 3.7 GHz Socket AM4 65W Desktop Processor - 100-100000065BOX', 'AMD Ryzen 7 5800X - Ryzen 7 5000 Series Vermeer (Zen 3) 8-Core 3.8 GHz Socket AM4 105W Desktop Processor - 100-100000063WOF', 'Intel Core i7-12700K - Core i7 12th Gen Alder Lake 12-Core (8P+4E) 3.6 GHz LGA 1700 125W Intel UHD Graphics 770 Desktop Processor - BX8071512700K', 'AMD Ryzen 9 5900X - Ryzen 9 5000 Series Vermeer (Zen 3) 12-Core 3.7 GHz Socket AM4 105W Desktop Processor - 100-100000061WOF', 'AMD Ryzen 5 5600G - Ryzen 5 5000 G-Series Cezanne (Zen 3) 6-Core 3.9 GHz Socket AM4 65W AMD Radeon Graphics Desktop Processor - 100-100000252BOX', 'Enermax M.2 2280 NVMe SSD Heatsink, Double-Sided Heat Sink, with Thermal Silicone Pad for PC / PS5 Cooling', 'Arctic Silver AS5-3.5G Thermal Compound', 'Intel Core i9-12900K - Core i9 12th Gen Alder Lake 16-Core (8P+8E) 3.2 GHz LGA 1700 125W Intel UHD Graphics 770 Desktop Processor - BX8071512900K', 'Intel Core i7-1170

In [5]:
# Create a dataframe to combine newegg's cpu models along with their prices

new_egg_cpu_df = pd.DataFrame()
new_egg_cpu_df['CPU'] = cpu_list
new_egg_cpu_df['Price'] = price_list
new_egg_cpu_df.head()

,CPU,Price
0,AMD Ryzen 5 5600X - Ryzen 5 5000 Series Vermee...,$321.50 –
1,AMD Ryzen 7 5800X - Ryzen 7 5000 Series Vermee...,$349.97 –
2,Intel Core i7-12700K - Core i7 12th Gen Alder ...,$374.98 –
3,AMD Ryzen 9 5900X - Ryzen 9 5000 Series Vermee...,$552.99 –
4,AMD Ryzen 5 5600G - Ryzen 5 5000 G-Series Ceza...,$265.00 –


In [6]:
# Extracting only the model # for each CPU, which can later be used to compare with the competitors' listings
# Extracting only numbers from the 'Price' columns (string - float)
# Drop the nulls

new_egg_cpu_df['CPU'] =new_egg_cpu_df['CPU'].str.split(' ', 0).str[-1].str[::1]
new_egg_cpu_df['Price'] = new_egg_cpu_df['Price'].str.extract('(\d+\.\d+)').astype(float)
new_egg_cpu_df=new_egg_cpu_df.dropna()
new_egg_cpu_df = new_egg_cpu_df.drop_duplicates()
new_egg_cpu_df

,CPU,Price
0,100-100000065BOX,321.50
1,100-100000063WOF,349.97
2,BX8071512700K,374.98
3,100-100000061WOF,552.99
4,100-100000252BOX,265.00
...,...,...
103,Compatible,79.99
104,White,294.99
105,Compatible,184.99
106,White,274.99


In [7]:
new_egg_cpu_df.to_csv('new_egg_cpu.csv', index = False)